In [2]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import re
import string
import nltk
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr, kendalltau
import gensim.downloader as api

C:\Users\utente\anaconda3\envs\text\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

# Data Importing and Sorting

In [4]:
files = os.listdir('corpus')
files.remove('.DS_Store')
files.remove('scores_ru-en.csv')
scaler = MinMaxScaler()
for file_ in files:
  name = file_.split('-')[0] + file_.split('-')[1]
  vars()[name] = pd.read_csv(os.path.join('corpus', file_, 'scores.csv'))
  vars()[name].drop(columns = ['source', 'annotators', 'avg-score'], inplace = True)
  vars()[name]['z-score'] = scaler.fit_transform(vars()[name]['z-score'].values.reshape(-1,1)) #normalizing values betwewen 0 and 1

In [5]:
english = csen.copy()
for df in [deen, ruen, zhen]:
  english = english.append(df)

In [6]:
finnish = enfi.copy()
chinese = enzh.copy()

In [7]:
english.reset_index(drop = True, inplace = True)

# Cleaning the corpus (updated  cleaning function)

In [8]:
def clean(text_list,
          lower = False,
          keep_numbers = False,
          keep_expression = False,
          remove_char = False,
          remove_stop = False,
          remove_tag = False,
          lemmatize = False,
          stemmer = False,
          english = True
          ):
    """
    Function that a receives a list of strings and preprocesses it.
    
    :param text_list: List of strings.
    :param lemmatize: Tag to apply lemmatization if True.
    :param stemmer: Tag to apply the stemmer if True.
    """
    if english:
        lang = 'english'
    else:
        lang = 'finnish'
    
    stop = set(stopwords.words(lang))
    stem = SnowballStemmer(lang)
    
    updates = []
    for j in range(len(text_list)):
        
        text = text_list[j]
        
        #LOWERCASE TEXT
        if lower:
            text = text.lower()
            
        #KEEP NUMBERS AS TOKENS
        if not keep_numbers:
            text = re.sub("[\d+]", 'X', text)
        
        #KEEP '?' and '!' AS TOKENS
        if not keep_expression:
            text = re.sub("[\?|\!]", 'EXPRESSION', text)
            
        #REMOVE TAGS
        if remove_tag:
            text = BeautifulSoup(text).get_text()
            
        #REMOVE THAT IS NOT TEXT
        if remove_char:
            text = re.sub("[^a-zA-Z]", ' ', text)
        
        #REMOVE STOP WORDS
        if remove_stop:
            text = ' '.join([word for word in text.split(' ') if word not in stop])
        
        #LEMMATIZATION
        if lemmatize:
            if english:
                lemma = WordNetLemmatizer()
                text = " ".join(lemma.lemmatize(word) for word in text.split())
        
        #STEMMER
        if stemmer:
            text = " ".join(stem.stem(word) for word in text.split())
        
        updates.append(text)
        
    return updates

def clean_ch(text_list, keep_numbers=False, remove_punctuation=False, remove_stop = False, stopwords_set='merged'):
    """
    Function that removes chinese stopwords
    
    :param stopwords_set: remove words of both sets (merged), just the 1st (fst) or just the second (snd) 
    """
    updates = []
    
    zh_stopwords1 = [line.strip() for line in open('chinese_stopwords/chinese_stopwords1.txt', 'r', encoding='utf-8').readlines()]
    zh_stopwords2 = [line.strip() for line in open('chinese_stopwords/chinese_stopwords2.txt', 'r', encoding='utf-8').readlines()]
    
    if stopwords_set == 'merged':
        stop = list(set(zh_stopwords1 + zh_stopwords2))
    elif stopwords_set == 'fst':
        stop = zh_stopwords1
    elif stopwords_set == 'snd':
        stop = zh_stopwords2

    for j in range(len(text_list)):
        
        text = text_list[j]
        
        #KEEP NUMBERS AS TOKENS
        if keep_numbers:
            text = re.sub("[\d+]", 'X', text)
        
        # REMOVE PUNCTUATION
        if remove_punctuation:
            # https://stackoverflow.com/questions/36640587/how-to-remove-chinese-punctuation-in-python
            punc = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏."
            text = re.sub(r"[%s]+" %punc, "", text)
        
        # REMOVE STOP WORDS
        if remove_stop:
            text = ' '.join([word for word in text.split(' ') if word not in stop])
            
        updates.append(text)
        
    return updates

In [9]:
finnish_cleaned = pd.DataFrame()
finnish_cleaned['z-score'] = finnish['z-score']
for column in ['reference', 'translation']:
    finnish_cleaned[column] = clean(finnish[column],
                                    lower = True,
                                    keep_numbers = False,
                                    keep_expression = True,
                                    remove_char = True,
                                    remove_stop = False,
                                    remove_tag = True,
                                    lemmatize = False,
                                    stemmer = True,
                                    english = False)

In [10]:
chinese_cleaned = pd.DataFrame()
chinese_cleaned['z-score'] = chinese['z-score']
for column in ['reference', 'translation']:
    chinese_cleaned[column] = clean_ch(chinese[column],
                                    keep_numbers = True,
                                    remove_punctuation = False,
                                    remove_stop = True,
                                    stopwords_set = 'merged')

# Train, Dev & Test Split

In [11]:
# en_train, en_dev = train_test_split(english, shuffle = True, test_size = 0.2, random_state = 7)
# en_dev, en_test = train_test_split(en_dev, shuffle = True, test_size = 0.5, random_state = 7)

# en_train_cleaned, en_dev_cleaned = train_test_split(english_cleaned, shuffle = True, test_size = 0.2, random_state = 7)
# en_dev_cleaned, en_test_cleaned = train_test_split(en_dev_cleaned, shuffle = True, test_size = 0.5, random_state = 7)

##NEED TO IMPLEMENT
fin_train, fin_dev = train_test_split(finnish, shuffle = True, test_size = 0.2, random_state = 7)
fin_dev, fin_test = train_test_split(fin_dev, shuffle = True, test_size = 0.5, random_state = 7)

fin_train_cleaned, fin_dev_cleaned = train_test_split(finnish_cleaned, shuffle = True, test_size = 0.2, random_state = 7)
fin_dev_cleaned, fin_test_cleaned = train_test_split(fin_dev_cleaned, shuffle = True, test_size = 0.5, random_state = 7)

ch_train, ch_dev = train_test_split(chinese, shuffle = True, test_size = 0.2, random_state = 7)
ch_dev, ch_test = train_test_split(ch_dev, shuffle = True, test_size = 0.5, random_state = 7)

ch_train_cleaned, ch_dev_cleaned = train_test_split(chinese_cleaned, shuffle = True, test_size = 0.2, random_state = 7)
ch_dev_cleaned, ch_test_cleaned = train_test_split(ch_dev_cleaned, shuffle = True, test_size = 0.5, random_state = 7)

# Not cleaned corpus

## Encoding (Word2Vec + Word Mover Distance)

In [12]:
model = api.load('word2vec-google-news-300')

In [14]:
names = ['fin_train', 'fin_dev', 'fin_test']
for j,df in enumerate([fin_train, fin_dev, fin_test]):
    name = 'distances_' + names[j]
    vars()[name] = []
    df.reset_index(drop = True, inplace = True)
    for i in tqdm(range(len(df))):
                  
        vars()[name].append(model.wmdistance(df['reference'][i], df['translation'][i]))
    
    name2 = 'score_' + names[j]
    vars()[name2] = np.array(df['z-score'])

  0%|          | 0/5398 [00:00<?, ?it/s]

  0%|          | 0/675 [00:00<?, ?it/s]

  0%|          | 0/675 [00:00<?, ?it/s]

In [15]:
names = ['ch_train', 'ch_dev', 'ch_test']
for j,df in enumerate([ch_train, ch_dev, ch_test]):
    name = 'distances_' + names[j]
    vars()[name] = []
    df.reset_index(drop = True, inplace = True)
    for i in tqdm(range(len(df))):
                  
        vars()[name].append(model.wmdistance(df['reference'][i], df['translation'][i]))
    
    name2 = 'score_' + names[j]
    vars()[name2] = np.array(df['z-score'])

  0%|          | 0/8176 [00:00<?, ?it/s]

  0%|          | 0/1022 [00:00<?, ?it/s]

  0%|          | 0/1023 [00:00<?, ?it/s]

## Calculating correlation

In [16]:
def corr(y_train_true, y_train_pred, y_dev_true, y_dev_pred, y_test_true, y_test_pred, return_corr = False):
    
    cleaned_corr_train, cleaned_corr_train_pvalue = pearsonr(y_train_true, y_train_pred)
    cleaned_corr_ktau_train, cleaned_corr_ktau_train_pvalue = kendalltau(y_train_true, y_train_pred)
    
    cleaned_corr_dev, cleaned_corr_dev_pvalue = pearsonr(y_dev_true, y_dev_pred)
    cleaned_corr_ktau_dev, cleaned_corr_ktau_dev_pvalue = kendalltau(y_dev_true, y_dev_pred)


    cleaned_corr_ktau_test, cleaned_corr_ktau_test_pvalue = kendalltau(y_test_true, y_test_pred)
    cleaned_corr_test, cleaned_corr_test_pvalue = pearsonr(y_test_true, y_test_pred)
        
    print(f'Pearson correlation between cosine similarity and score on train set: {cleaned_corr_train} (p-value < 0.001: {cleaned_corr_train_pvalue < 0.001}); and Kendall Tau: {cleaned_corr_ktau_train} (p-value < 0.001: {cleaned_corr_ktau_train_pvalue < 0.001})')
    print(f'Pearson correlation between cosine similarity and score on development set: {cleaned_corr_dev} (p-value < 0.001: {cleaned_corr_dev_pvalue < 0.001}); and Kendall Tau: {cleaned_corr_ktau_dev} (p-value < 0.001: {cleaned_corr_ktau_dev_pvalue < 0.001})')
    print(f'Pearson correlation between cosine similarity and score on test set: {cleaned_corr_test} (p-value < 0.001: {cleaned_corr_test_pvalue < 0.001}); and Kendall Tau: {cleaned_corr_ktau_test} (p-value < 0.001: {cleaned_corr_ktau_test_pvalue < 0.001})')
    
    if return_corr:
        return cleaned_corr_dev

How to treat np.inf???

In [17]:
while np.inf in distances_fin_train:
    distances_fin_train[distances_fin_train.index(np.inf)] = 10

while np.inf in distances_fin_dev:
    distances_fin_dev[distances_fin_dev.index(np.inf)] = 10
    
while np.inf in distances_fin_test:
    distances_fin_test[distances_fin_test.index(np.inf)] = 10

In [18]:
while np.inf in distances_ch_train:
    distances_ch_train[distances_ch_train.index(np.inf)] = 10

while np.inf in distances_ch_dev:
    distances_ch_dev[distances_ch_dev.index(np.inf)] = 10
    
while np.inf in distances_ch_test:
    distances_ch_test[distances_ch_test.index(np.inf)] = 10

In [19]:
corr(score_fin_train, distances_fin_train, score_fin_dev, distances_fin_dev, score_fin_test, distances_fin_test)

Pearson correlation between cosine similarity and score on train set: -0.4423001770601065 (p-value < 0.001: True); and Kendall Tau: -0.304006891633581 (p-value < 0.001: True)
Pearson correlation between cosine similarity and score on development set: -0.40423504312735 (p-value < 0.001: True); and Kendall Tau: -0.2822384195577719 (p-value < 0.001: True)
Pearson correlation between cosine similarity and score on test set: -0.35667416940720503 (p-value < 0.001: True); and Kendall Tau: -0.25342281135122874 (p-value < 0.001: True)


In [20]:
corr(score_ch_train, distances_ch_train, score_ch_dev, distances_ch_dev, score_ch_test, distances_ch_test)

Pearson correlation between cosine similarity and score on train set: -0.25288504707134435 (p-value < 0.001: True); and Kendall Tau: -0.24110220002375687 (p-value < 0.001: True)
Pearson correlation between cosine similarity and score on development set: -0.25864761576879836 (p-value < 0.001: True); and Kendall Tau: -0.25073979422608195 (p-value < 0.001: True)
Pearson correlation between cosine similarity and score on test set: -0.32907919454326545 (p-value < 0.001: True); and Kendall Tau: -0.25310354525228784 (p-value < 0.001: True)


# Cleaned corpus

## Encoding (Word2Vec + Word Mover Distance)

In [21]:
# model = api.load('word2vec-google-news-300')
names = ['fin_train_cleaned', 'fin_dev_cleaned', 'fin_test_cleaned']
for j,df in enumerate([fin_train_cleaned, fin_dev_cleaned, fin_test_cleaned]):
    name = 'distances_' + names[j]
    vars()[name] = []
    df.reset_index(drop = True, inplace = True)
    for i in tqdm(range(len(df))):
                  
        vars()[name].append(model.wmdistance(df['reference'][i], df['translation'][i]))
    
    name2 = 'score_' + names[j]
    vars()[name2] = np.array(df['z-score'])

  0%|          | 0/5398 [00:00<?, ?it/s]

  0%|          | 0/675 [00:00<?, ?it/s]

  0%|          | 0/675 [00:00<?, ?it/s]

In [23]:
# model = api.load('word2vec-google-news-300')
names = ['ch_train_cleaned', 'ch_dev_cleaned', 'ch_test_cleaned']
for j,df in enumerate([ch_train_cleaned, ch_dev_cleaned, ch_test_cleaned]):
    name = 'distances_' + names[j]
    vars()[name] = []
    df.reset_index(drop = True, inplace = True)
    for i in tqdm(range(len(df))):
                  
        vars()[name].append(model.wmdistance(df['reference'][i], df['translation'][i]))
    
    name2 = 'score_' + names[j]
    vars()[name2] = np.array(df['z-score'])

  0%|          | 0/8176 [00:00<?, ?it/s]

  0%|          | 0/1022 [00:00<?, ?it/s]

  0%|          | 0/1023 [00:00<?, ?it/s]

## Calculating correlation

In [24]:
while np.inf in distances_fin_train_cleaned:
    distances_fin_train_cleaned[distances_fin_train_cleaned.index(np.inf)] = 10

while np.inf in distances_fin_dev_cleaned:
    distances_fin_dev_cleaned[distances_fin_dev_cleaned.index(np.inf)] = 10
    
while np.inf in distances_fin_test_cleaned:
    distances_fin_test_cleaned[distances_fin_test_cleaned.index(np.inf)] = 10

In [25]:
while np.inf in distances_ch_train_cleaned:
    distances_ch_train_cleaned[distances_ch_train_cleaned.index(np.inf)] = 10

while np.inf in distances_ch_dev_cleaned:
    distances_ch_dev_cleaned[distances_ch_dev_cleaned.index(np.inf)] = 10
    
while np.inf in distances_ch_test_cleaned:
    distances_ch_test_cleaned[distances_ch_test_cleaned.index(np.inf)] = 10

In [26]:
corr(score_fin_train_cleaned, distances_fin_train_cleaned, score_fin_dev_cleaned, distances_fin_dev_cleaned, score_fin_test_cleaned, distances_fin_test_cleaned)

Pearson correlation between cosine similarity and score on train set: -0.43161416179470985 (p-value < 0.001: True); and Kendall Tau: -0.29760017477818435 (p-value < 0.001: True)
Pearson correlation between cosine similarity and score on development set: -0.3815949530749775 (p-value < 0.001: True); and Kendall Tau: -0.27013049060618927 (p-value < 0.001: True)
Pearson correlation between cosine similarity and score on test set: -0.3367278955650935 (p-value < 0.001: True); and Kendall Tau: -0.24948405560974188 (p-value < 0.001: True)


In [27]:
corr(score_ch_train_cleaned, distances_ch_train_cleaned, score_ch_dev_cleaned, distances_ch_dev_cleaned, score_ch_test_cleaned, distances_ch_test_cleaned)

Pearson correlation between cosine similarity and score on train set: -0.2564296863991978 (p-value < 0.001: True); and Kendall Tau: -0.24550555280016723 (p-value < 0.001: True)
Pearson correlation between cosine similarity and score on development set: -0.2592715719253612 (p-value < 0.001: True); and Kendall Tau: -0.2541648397139081 (p-value < 0.001: True)
Pearson correlation between cosine similarity and score on test set: -0.3341068811315575 (p-value < 0.001: True); and Kendall Tau: -0.255442594880196 (p-value < 0.001: True)
